## mask_buildings
### Remove information for buildings deemed too sensitive (e.g. Military)

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "DB2_CONNECTION_STRING": "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;",
    "DB2_USERNAME": "xxx",
    "DB2_PASSWORD": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "COUNTRY_TABLE": "FEATURES_DB_MAHARASHTRA"
    }
    """


In [4]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [ ]:
#Install neccessary packages if not installed before
#! pip install shapely
#! pip install JayDeBeApi

In [5]:
# Import necessary libraries
import pandas as pd
import overpass
import geojson
import requests
import getpass
from shapely.geometry import Point, Polygon, mapping, shape
import shapely
from shapely import wkt
import json
# import geopandas as gpd
import jaydebeapi as jdbc
import jpype
import ibm_boto3
import gc
import io
import os
from botocore.client import Config
from tqdm import tqdm
from skimage import measure as M

In [6]:
sql_tablename = config["COUNTRY_TABLE"]

In [7]:
#load db2jcc4.jar
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                              ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])

response = cos_client.list_objects_v2(Bucket=config["UTILS_BUCKET"])

try:
    for obj in response['Contents']:
        name = obj['Key']
        
        if name == 'db2jcc4.jar':
            streaming_body_1 = cos_client.get_object(Bucket=config["UTILS_BUCKET"], Key=name)['Body']
            print("Copying to localStorage :  " + name)
            with io.FileIO(name, 'w') as file:
                for i in io.BytesIO(streaming_body_1.read()):
                    file.write(i)
    
    from utils import *
    print('External utils succesfully imported')
except Exception as e:
    print('Error occured: ', e)

Copying to localStorage :  db2jcc4.jar
Error occured:  No module named 'utils'


In [8]:

# connect to the IBM DB2 function
def connect_to_db():

    jar = 'db2jcc4.jar'
    os.environ['CLASSPATH'] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    # create JDBC connection
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                config['DB2_CONNECTION_STRING'],
                [config["DB2_USERNAME"], config["DB2_PASSWORD"]],
                'db2jcc4.jar')
    
    return conn

DB2_connection = connect_to_db()
cursor = DB2_connection.cursor()

/var/folders/42/bkr77x_n1qv118pt28lzd3380000gn/T/ipykernel_43250/2319750877.py:14: DeprecationWarning: jpype._core.isThreadAttachedToJVM is deprecated, use java.lang.Thread.isAttached instead
  if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():


In [9]:
def fetch_inactive_builings():
    '''
        This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    '''

    # fetch column names from defined SQL table

    columns = ['latitude', 'longitude']
    
    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{sql_tablename}
        WHERE 
            "OSM_TYPE" = 'Inactive'
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        df = pd.DataFrame(data=data, columns=columns)

        return df

In [16]:
def update_DB2_row(lat, lon, cursor):
    try:
        
        if sql_tablename == 'FEATURES_DB_MAHARASHTRA':
            sql = f"""
            UPDATE "USER1"."{sql_tablename}"
                SET
                    "ML_MODEL" = '-',
                    "ML_CONFIDENCE" = NULL,
                    "HEIGHT" = NULL,
                    "HEIGHT_MEDIAN" = NULL,
                    "HEIGHT_MEAN" = NULL,
                    "HEIGHT_MAX" = NULL,
                    "URBAN_SPLIT" = NULL,
                    "GHSL_SMOD" = NULL,
                    "FLOORS" = NULL,
                    "GFA_IN_METERS" = NULL,
                    "CLASSIFICATION_TYPE" = 'Inactive'
                WHERE 
                    ("LATITUDE" = {lat}) AND 
                    ("LONGITUDE" = {lon})
            """
            
        elif sql_tablename == 'FEATURES_DB_VIDA_EXTENDED':
            sql = f"""
            UPDATE "USER1"."{sql_tablename}"
                SET
                    "ML_MODEL" = '-',
                    "ML_CONFIDENCE" = NULL,
                    "HEIGHT" = NULL,
                    "HEIGHT_MEDIAN" = NULL,
                    "HEIGHT_MEAN" = NULL,
                    "HEIGHT_MAX" = NULL,
                    "URBAN_SPLIT" = NULL,
                    "GHSL_SMOD" = NULL,
                    "FLOORS" = NULL,
                    "GFA_IN_METERS" = NULL,
                    "ELEC_ACCESS_PERCENT" = NULL,
                    "ELEC_CONSUMPTION_KWH_MONTH" = NULL,
                    "ELEC_CONSUMPTION_STD_KWH_MONTH" = NULL,
                    "CLASSIFICATION_TYPE" = 'Inactive'
                WHERE 
                    ("LATITUDE" = {lat}) AND 
                    ("LONGITUDE" = {lon})
            """
            
        cursor.execute(sql)
    except Exception as e:
        print(e, sql)
        

In [15]:
df = fetch_inactive_builings()
df

Fetch items error occured: cannot access local variable 'cursor' where it is not associated with a value
Reconnecting to the database try again...
startJVM exception:  JVM is already started


,latitude,longitude
0,0.599188,34.620904
1,0.598779,34.621072
2,0.598905,34.621080
3,0.599217,34.621081
4,0.598582,34.621095
...,...,...
8344,-4.078261,39.629762
8345,-4.077238,39.629797
8346,-4.082093,39.629827
8347,-4.073610,39.629852


In [14]:
conn = connect_to_db()
cursor = conn.cursor()

startJVM exception:  JVM is already started


In [13]:
for row in df.itertuples():
    
    try:
        update_DB2_row(row.latitude, row.longitude, cursor)
    except Exception as e:
        
        conn = connect_to_db()
        cursor = conn.cursor()
        update_DB2_row(row.latitude, row.longitude, cursor)
